In [36]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import regexp_replace, lower, col, udf
import pyspark.sql.functions as f
from pyspark.ml.feature import Tokenizer, HashingTF, StopWordsRemover, IDF
from pyspark.ml.classification import NaiveBayes 
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [2]:
spark = SparkSession.builder.master("local[*]") \
.appName('MyApplicationName') \
.getOrCreate()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [3]:
sc.version

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

'2.4.8-amzn-2'

In [3]:
df = spark.read.json("s3://my-data-bucket-yjh/yelp_academic_dataset_review.json")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [5]:
df.show(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+----+-------------------+-----+--------------------+-----+--------------------+------+--------------------+
|         business_id|cool|               date|funny|           review_id|stars|                text|useful|             user_id|
+--------------------+----+-------------------+-----+--------------------+-----+--------------------+------+--------------------+
|XQfwVwDr-v0ZS3_Cb...|   0|2018-07-07 22:09:11|    0|KU_O5udG6zpxOg-Vc...|  3.0|If you decide to ...|     0|mh_-eMZ6K5RLWhZyI...|
|7ATYjTIgM3jUlt4UM...|   1|2012-01-03 15:28:18|    0|BiTunyQ73aT9WBnpR...|  5.0|I've taken a lot ...|     1|OyoGAe7OKpv6SyGZT...|
|YjUWPpI6HXG530lwP...|   0|2014-02-05 20:30:30|    0|saUsX_uimxRlCVr67...|  3.0|Family diner. Had...|     0|8g_iMtfSiwikVnbP2...|
|kxX2SOes4o-D3ZQBk...|   1|2015-01-04 00:01:03|    0|AqPFMleE6RsU23_au...|  5.0|Wow!  Yummy, diff...|     1|_7bHUi9Uuf5__HHc_...|
|e4Vwtrqf-wpJfwesg...|   1|2017-01-14 20:54:15|    0|Sx8TMOWLNuJBWer-0...|  4.0|Cute inter

In [7]:
print(f'The data set has {df.count()} number of records, and {len(df.columns)} columns')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

The data set has 6990280 number of records, and 9 columns

In [8]:
df.dtypes

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

[('business_id', 'string'), ('cool', 'bigint'), ('date', 'string'), ('funny', 'bigint'), ('review_id', 'string'), ('stars', 'double'), ('text', 'string'), ('useful', 'bigint'), ('user_id', 'string')]

In [25]:
df.summary().show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

An error was encountered:
'NoneType' object has no attribute 'round'
Traceback (most recent call last):
AttributeError: 'NoneType' object has no attribute 'round'



In [11]:
df.where(df['text'].isNull()).count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

0

In [18]:
df.select(df['text']).distinct().count() == df.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

False

In [16]:
df.dropDuplicates().show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+----+-------------------+-----+--------------------+-----+--------------------+------+--------------------+
|         business_id|cool|               date|funny|           review_id|stars|                text|useful|             user_id|
+--------------------+----+-------------------+-----+--------------------+-----+--------------------+------+--------------------+
|eL4lyE7LNoXEMvpcJ...|   0|2011-08-25 16:14:24|    0|dpgq69tsI2f_GpRH7...|  3.0|This was a tough ...|     5|Uk3X2AypU8AqvcYEV...|
|jsUzVBGEIoer6l_fx...|   0|2014-03-15 02:46:41|    0|I7XGtAW-KkzVDvtCv...|  1.0|The worker was aw...|     6|ulSJonL748r7a1W-t...|
|I8eSWVzc_y9ORuKWg...|   0|2016-02-12 02:10:06|    0|EwToghmaFs7w5vwyO...|  5.0|Despina is the wa...|     1|2_yTdb1lyVQFwL8tA...|
|34Eqv8jXgxg_EEwcs...|   0|2016-10-26 17:03:18|    0|J8KTnbCtArCsPKynC...|  5.0|Bomb. I go for th...|     0|RMmcPwhkkivzm80a2...|
|GBTPC53ZrG1ZBY3DT...|   1|2011-09-25 20:00:29|    1|30NhLcomHTXBRoXY8...|  5.0|If you are

In [4]:
df = df.dropDuplicates(subset=['text'])
print(df.count())

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

6974127

In [22]:
df.groupBy('stars').count().show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----+-------+
|stars|  count|
+-----+-------+
|  1.0|1066233|
|  5.0|3223620|
|  2.0| 543258|
|  4.0|1450232|
|  3.0| 690784|
+-----+-------+

In [39]:
df = df.withColumn('wordCount', f.size(f.split(f.col('text'), ' ')))
df.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+----+-------------------+-----+--------------------+-----+--------------------+------+--------------------+---------+
|         business_id|cool|               date|funny|           review_id|stars|                text|useful|             user_id|wordCount|
+--------------------+----+-------------------+-----+--------------------+-----+--------------------+------+--------------------+---------+
|Cjwb7VQGP0u2eWYj1...|   0|2017-08-25 05:46:31|    0|eKI6tX2_8BFUYtDA8...|  1.0|!!! DO NOT COME H...|     7|kuzLeLM0DVrcWToFK...|      631|
|szcdtj3XvJIVKLNjh...|   0|2015-09-10 18:45:38|    0|ZbCkCUgQVIq75h-Fj...|  1.0|!!!!!!BE AWARE!!!...|     0|1QftliNjMf7FTCd2j...|      320|
|GbGbh15JQXb0-V-XY...|   1|2013-09-23 00:13:47|    3|KmXSSCHs3rnfg6Fe1...|  5.0|!Clarifier - this...|     6|dBrQG46MhVWobyPeE...|      169|
|n4XX_Im7-njZ8YCdw...|   0|2015-02-08 17:45:00|    1|zHZ2Wh-c3BpGPPBfZ...|  1.0|" after a previou...|     4|7lNgj9Wv999fSPV_m...|      132|
|a32M9H9nkkPGKTKB5..

In [30]:
df = df.withColumn('text', regexp_replace(df.text, '[_():;,.!""?\\$&''-]', ' '))
df = df.withColumn('text', regexp_replace(df.text, '[0-9]', ' '))
df = df.withColumn('text', regexp_replace(df.text, ' +', ' '))
df = df.withColumn('text', lower(df.text))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [22]:
df = Tokenizer(inputCol='text', outputCol='words').transform(df)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [26]:
df = StopWordsRemover(inputCol='words', outputCol='terms').transform(df)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

An error was encountered:
'requirement failed: Column terms already exists.'
Traceback (most recent call last):
  File "/usr/lib/spark/python/lib/pyspark.zip/pyspark/ml/base.py", line 173, in transform
    return self._transform(dataset)
  File "/usr/lib/spark/python/lib/pyspark.zip/pyspark/ml/wrapper.py", line 312, in _transform
    return DataFrame(self._java_obj.transform(dataset._jdf), dataset.sql_ctx)
  File "/usr/lib/spark/python/lib/py4j-0.10.7-src.zip/py4j/java_gateway.py", line 1257, in __call__
    answer, self.gateway_client, self.target_id, self.name)
  File "/usr/lib/spark/python/lib/pyspark.zip/pyspark/sql/utils.py", line 79, in deco
    raise IllegalArgumentException(s.split(': ', 1)[1], stackTrace)
pyspark.sql.utils.IllegalArgumentException: 'requirement failed: Column terms already exists.'



In [40]:
df = HashingTF(inputCol='terms', outputCol='hash').transform(df)
tf_idf = IDF(inputCol='hash', outputCol='features').fit(df).transform(df)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

An error was encountered:
'requirement failed: Column hash already exists.'
Traceback (most recent call last):
  File "/usr/lib/spark/python/lib/pyspark.zip/pyspark/ml/base.py", line 173, in transform
    return self._transform(dataset)
  File "/usr/lib/spark/python/lib/pyspark.zip/pyspark/ml/wrapper.py", line 312, in _transform
    return DataFrame(self._java_obj.transform(dataset._jdf), dataset.sql_ctx)
  File "/usr/lib/spark/python/lib/py4j-0.10.7-src.zip/py4j/java_gateway.py", line 1257, in __call__
    answer, self.gateway_client, self.target_id, self.name)
  File "/usr/lib/spark/python/lib/pyspark.zip/pyspark/sql/utils.py", line 79, in deco
    raise IllegalArgumentException(s.split(': ', 1)[1], stackTrace)
pyspark.sql.utils.IllegalArgumentException: 'requirement failed: Column hash already exists.'



In [41]:
tf_idf.select('terms', 'features')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

DataFrame[terms: array<string>, features: vector]

In [29]:
df_train, df_test = df.randomSplit([0.8, 0.2], seed=13)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [31]:
df.show(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+----+-------------------+-----+--------------------+-----+--------------------+------+--------------------+--------------------+--------------------+--------------------+
|         business_id|cool|               date|funny|           review_id|stars|                text|useful|             user_id|               words|               terms|                hash|
+--------------------+----+-------------------+-----+--------------------+-----+--------------------+------+--------------------+--------------------+--------------------+--------------------+
|Cjwb7VQGP0u2eWYj1...|   0|2017-08-25 05:46:31|    0|eKI6tX2_8BFUYtDA8...|  1.0| do not come here...|     7|kuzLeLM0DVrcWToFK...|[, do, not, come,...|[, come, come, un...|(262144,[14,1558,...|
|szcdtj3XvJIVKLNjh...|   0|2015-09-10 18:45:38|    0|ZbCkCUgQVIq75h-Fj...|  1.0| be aware i went ...|     0|1QftliNjMf7FTCd2j...|[, be, aware, i, ...|[, aware, went, p...|(262144,[3530,523...|
|GbGbh15JQXb0-V-XY...|   1|2013-09-

In [42]:
nb = NaiveBayes(modelType="multinomial")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…